In [5]:
import great_expectations as gx
import pandas as pd
import warnings
warnings.filterwarnings("ignore", message="`result_format` configured at the Validator-level*")

# Load the data
df = pd.read_csv("/Users/jonaslarsson/PycharmProjects/Bank_Projekt/data/sebank_customers_with_accounts.csv")

# Create the ephemeral GX context
context = gx.get_context()

# Add a pandas datasource
data_source = context.data_sources.add_pandas(name="pandas")

# Add a dataframe asset
data_asset = data_source.add_dataframe_asset(name="customers_with_accounts_data")

# Define the batch (entire DataFrame)
batch_definition = data_asset.add_batch_definition_whole_dataframe(name="batch_def")
batch = batch_definition.get_batch(batch_parameters={"dataframe": df})

# Create the expectation suite with a name
suite = gx.core.expectation_suite.ExpectationSuite(name="customers_with_accounts_suite")

# Get the validator using the suite
validator = context.get_validator(batch=batch, expectation_suite=suite)

# Add expectations:
validator.expect_column_values_to_be_unique("BankAccount") # Kollar att alla kontonummer är unika
validator.expect_column_values_to_not_be_null("BankAccount") # kollar att inget kontonummer är NULL

validator.expect_column_values_to_match_regex("BankAccount", r"^SE8902.*$") #kollar att bankens alla konton börjar på SE8902



# Validate
results = validator.validate()

# Print results
print(results)

Calculating Metrics: 100%|██████████| 12/12 [00:00<00:00, 2120.48it/s]

{
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_unique",
        "kwargs": {
          "batch_id": "pandas-customers_with_accounts_data",
          "column": "BankAccount"
        },
        "meta": {}
      },
      "result": {
        "element_count": 1000,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "missing_count": 0,
        "missing_percent": 0.0,
        "unexpected_percent_total": 0.0,
        "unexpected_percent_nonmissing": 0.0
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "batch_id": "pandas-customers_with_accounts_data",
          "column": "BankAcco